In [4]:
import numpy as np
import requests
import urllib.request
import progressbar
import pickle
import os

from subprocess import call
from multiprocessing import Pool
from bs4 import BeautifulSoup

In this cell we should write some code to get all links to a scriptie page. The cell below can then get the meta
data from this page and the pdf itself and convert this. 

http://www.scriptiesonline.uba.uva.nl/cgi/b/bib/bib-idx?c=uvascript;sid=987112f770cbd4ff253813ee78b67a20;lang=en;type=simple;rgn1=entire+record;q1=*

In [25]:
def getLinks(url):
    allLinks = []
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    linksOnPage = soup.findAll(class_='detaillink')
    for link in linksOnPage:
        allLinks += [link.get('href')]
        print(link.get('href'))
url = 'http://www.scriptiesonline.uba.uva.nl/cgi/b/bib/bib-idx?c=uvascript;sid=987112f770cbd4ff253813ee78b67a20;lang=en;type=simple;rgn1=entire+record;q1=*'
getLinks(url)
# Dus jullie denken nu misschien: hey, dit is helemaal niet af. Maar het leuke is dat er in die links
# die hieronder staan zo'n mooie variabele zit; start = *. Deze start is gewoon een getalletje dat optelt over
# alle scripties. Vul je 45715 in krijg je de aller laatste scriptie. 

# zo kun je vrij gemakkelijk een lijstje maken voor alle scriptie paginas, meta data stelen en dan de pdf downloaden!

http://www.scriptiesonline.uba.uva.nl/cgi/b/bib/bib-idx?c=uvascript;sid=987112f770cbd4ff253813ee78b67a20;lang=en;type=simple;rgn1=entire%20record;q1=%2A;sort=publicationyear;cc=uvascript;view=reslist;fmt=long;page=reslist;start=1;size=1
http://www.scriptiesonline.uba.uva.nl/cgi/b/bib/bib-idx?c=uvascript;sid=987112f770cbd4ff253813ee78b67a20;lang=en;type=simple;rgn1=entire%20record;q1=%2A;sort=publicationyear;cc=uvascript;view=reslist;fmt=long;page=reslist;start=2;size=1
http://www.scriptiesonline.uba.uva.nl/cgi/b/bib/bib-idx?c=uvascript;sid=987112f770cbd4ff253813ee78b67a20;lang=en;type=simple;rgn1=entire%20record;q1=%2A;sort=publicationyear;cc=uvascript;view=reslist;fmt=long;page=reslist;start=3;size=1
http://www.scriptiesonline.uba.uva.nl/cgi/b/bib/bib-idx?c=uvascript;sid=987112f770cbd4ff253813ee78b67a20;lang=en;type=simple;rgn1=entire%20record;q1=%2A;sort=publicationyear;cc=uvascript;view=reslist;fmt=long;page=reslist;start=4;size=1
http://www.scriptiesonline.uba.uva.nl/cgi/b/bib/bib-

Let's start with getting the meta-data from the scriptie pages.

Example page: 

http://www.scriptiesonline.uba.uva.nl/cgi/b/bib/bib-idx?c=uvascript;sid=987112f770cbd4ff253813ee78b67a20;lang=en;type=simple;rgn1=entire%20record;q1=%2A;sort=publicationyear;cc=uvascript;view=reslist;fmt=long;page=reslist;start=1;size=1

In [7]:
def getMeta(url):
    metaData = {}
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    keys = soup.findAll(class_='label')
    values = soup.findAll(class_='value')
    for key, value in zip(keys, values):
        metaData[key.text] = value.text
    try:
        # put whole link to download instead of just document/number
        metaData['Download'] = 'http://www.scriptiesonline.uba.uva.nl/' + metaData['Download']
    except:
        pass
    return metaData

url = 'http://www.scriptiesonline.uba.uva.nl/cgi/b/bib/bib-idx?c=uvascript;sid=987112f770cbd4ff253813ee78b67a20;lang=en;type=simple;rgn1=entire%20record;q1=%2A;sort=publicationyear;cc=uvascript;view=reslist;fmt=long;page=reslist;start=1;size=1'
data = getMeta(url)
print(data)
# TODO: Een probleem hiermee is nog dat er <br> in de tekst van het abstract staat om een nieuwe regel te beginnen.
# Mooie Soep gooit deze weg, met als gevolg dat er geen spatie tussen die tekst staat en er tweede woorden aan elkaar
# plakken, hier moet nog iets aan gedaan worden.

# TODO2: link naar document wordt nu een beetje raar gemaakt in de bovenstaande try except, soms werkt dit niet. 
# Eigenlijk moet de url uit de href gehaald worden, ipv de tekst die klikbaar is.

{'Title': 'Subliminal Priming towards healthy Food choice: The Moderating Effect of Working Memory Capacity', 'Supervisor': 'Tina DudenhÃ¶ffer', 'Abstract': 'Food advertisement has a big impact on the eating behaviour of consumers and policy makers increasingly focus on advertisement for healthier food selection by consumers. Although priming towards healthier food choice has been explored, little research has beendone on priming towards Healthier Food Choice at a subliminal level. To determine how Working Memory Capacity (WMC) influences consumers to be subliminally primed towards healthier food, this research combines the fields of cognitive research and consumer behaviour. This research proposed that Subliminal Priming positively influences consumersâ\x80\x99Healthy Food Choice. More specifically, it hypothesises that WMC has a positive moderating effect on the relationship between Subliminal Priming and Healthy Food Choice, if this is in line with the current goals of the consumer.

In [64]:
# non multithreaded version, rather slow.
def getPDFtoTXTtester(totalAmount, threadID):
    # make list from 1 to and including totalAmount
    allNumbers = np.arange(totalAmount + 1)[1:]
    for number in allNumbers:
        url = 'http://www.scriptiesonline.uba.uva.nl/cgi/b/bib/bib-idx?c=uvascript;sid=987112f770cbd4ff253813ee78b67a20;lang=en;type=simple;rgn1=entire%20record;q1=%2A;sort=publicationyear;cc=uvascript;view=reslist;fmt=long;page=reslist;start=' + str(number) + ';size=1'
        metaData = getMeta(url)
        fileName = "/home/douwe/Downloads/pdfparsing/" +  str(threadID) + metaData['Download'].split("/")[-1] + '.pdf'
        with open(fileName[:-4]+ '.pkl', 'wb') as handle:
            pickle.dump(metaData, handle, protocol=pickle.HIGHEST_PROTOCOL)
        # Download the file from `url` and save it locally under `file_name`:
        urllib.request.urlretrieve(metaData['Download'], fileName)
        txtFileName = fileName[:-4] + '.txt'
        # call the java program
        call(["java", "-jar", "pdfbox-app-2.0.7.jar", "ExtractText", fileName, "-o", txtFileName])
        # remove the pdf again, since it's big and stupid.
        os.remove(fileName)
getPDFtoTXTtester(5, 1)

In [ ]:
# Multithreading getPDFtoTXT
amountScripties = 45715
amountThreads = 12
# devide a list in num equally long parts. Used for multiprocessing
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0
    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    return out

def getPDFtoTXTwrapper(allNumbers, threadID):
    skipped = []
    for number in allNumbers:
        # try except to ignore error things without fully crashing
        if(number % 200) == 0:
            print("thread", threadID, "is at: ", number, "working towards: ", allNumbers[-1])
        try:
            getPDFtoTXT(number, threadID)
        except:
            skipped += [number]
            pass
    with open("/home/douwe/Downloads/pdfskips/" + str(threadID) + "skipped.pkl", 'wb') as handle:
        pickle.dump(skipped, handle, protocol=pickle.HIGHEST_PROTOCOL)

def getPDFtoTXT(number, threadID):
    url = 'http://www.scriptiesonline.uba.uva.nl/cgi/b/bib/bib-idx?c=uvascript;sid=987112f770cbd4ff253813ee78b67a20;lang=en;type=simple;rgn1=entire%20record;q1=%2A;sort=publicationyear;cc=uvascript;view=reslist;fmt=long;page=reslist;start=' + str(number) + ';size=1'
    metaData = getMeta(url)
    fileName = "/home/douwe/Downloads/pdfdata/" +  str(threadID) + metaData['Download'].split("/")[-1] + '.pdf'
    with open(fileName[:-4]+ '.pkl', 'wb') as handle:
        pickle.dump(metaData, handle, protocol=pickle.HIGHEST_PROTOCOL)
    # Download the file from `url` and save it locally under `file_name`:
    urllib.request.urlretrieve(metaData['Download'], fileName)
    txtFileName = fileName[:-4] + '.txt'
    # call the java program
    call(["java", "-jar", "pdfbox-app-2.0.7.jar", "ExtractText", fileName, "-o", txtFileName])
    # remove the pdf again, since it's big and stupid.
    os.remove(fileName)
#getPDFtoTXTtester(5, 1)

allnumbers = np.arange(amountScripties + 1)[1:]
threadIDS = np.arange(amountThreads)
subListsNumbers = chunkIt(allnumbers, amountThreads)

#create a process Pool with 2 processes
pool1 = Pool(processes = amountThreads)

#map doWork to availble Pool processes
results = pool1.starmap(getPDFtoTXTwrapper, zip(tuple(subListsNumbers), threadIDS))

pool1.close()

thread 11 is at:  42000 working towards:  45715
thread 10 is at:  38200 working towards:  41905
thread 9 is at:  34400 working towards:  38095
thread 8 is at:  30600 working towards:  34286
thread 7 is at:  26800 working towards:  30476
thread 6 is at:  23000 working towards:  26667
thread 5 is at:  19200 working towards:  22857
thread 4 is at:  15400 working towards:  19047
thread 2 is at:  7800 working towards:  11428
thread 3 is at:  11600 working towards:  15238
thread 1 is at:  4000 working towards:  7619
thread 0 is at:  200 working towards:  3809
thread 11 is at:  42200 working towards:  45715
thread 8 is at:  30800 working towards:  34286
thread 10 is at:  38400 working towards:  41905
thread 9 is at:  34600 working towards:  38095
thread 7 is at:  27000 working towards:  30476
thread 6 is at:  23200 working towards:  26667
thread 5 is at:  19400 working towards:  22857
thread 4 is at:  15600 working towards:  19047
thread 2 is at:  8000 working towards:  11428
thread 3 is at: 

In [65]:
# load a pickle to see metadata
with open('/home/douwe/Downloads/pdfparsing/1636897.pkl', 'rb') as handle:
    meta = pickle.load(handle)
print(meta)

{'Year': '2016', 'Programme': 'FEB BSc Economie en Bedrijfskunde', 'Supervisor': 'O. Furtuna', ' Author': 'B. van den Berg', 'Download': 'http://www.scriptiesonline.uba.uva.nl/document/636897', 'Pages': '35', 'Abstract': 'In this paper the existence of a political budget cycle in The Netherlands from 1971 to 2014 is being researched. OLS regression for time-series is used. The dependent variables are the budget deficit as percentage of GDP and the payment on social securities as percentage of GDP. Controlling for other factors, no evidence was found for an increase of the budget deficit or the social security payments during election time. Furthermore, no evidence for a decrease of both dependent variables after elections was found.', 'Faculty': 'Faculty of Economics and Business', 'Document type': ' scriptie bachelor', 'Title': 'Every advantage has its disadvantage : does the right to vote introduce politicians acting selfish to stay in office?'}
